In [9]:
!pip install mlflow boto3 awscli

In [10]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/")

In [11]:
# Set or create an experiment
mlflow.set_experiment("Exp 3 - TfIdf Trigram max_features")

<Experiment: artifact_location='s3://mlflow-bucket-1218/572389916107840513', creation_time=1752576298945, experiment_id='572389916107840513', last_update_time=1752576298945, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram max_features', tags={}>

In [14]:
# Setup
import os
import time
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [13]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [15]:
# Set timeout environment variable
os.environ["MLFLOW_HTTP_REQUEST_TIMEOUT"] = "180"

# Set MLflow tracking URI
mlflow.set_tracking_uri("http://ec2-18-215-180-118.compute-1.amazonaws.com:5000")

# Function to run experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)

    # TF-IDF Vectorizer
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train, X_test, y_train, y_test = train_test_split(
        df['clean_comment'], df['category'],
        test_size=0.2, random_state=42, stratify=df['category']
    )
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    with mlflow.start_run() as run:
        # Set tags
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Train model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log fewer classification report metrics
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        mlflow.log_metric("f1_macro", classification_rep["macro avg"]["f1-score"])
        mlflow.log_metric("f1_weighted", classification_rep["weighted avg"]["f1-score"])
        mlflow.log_metric("precision_macro", classification_rep["macro avg"]["precision"])
        mlflow.log_metric("recall_macro", classification_rep["macro avg"]["recall"])

        # Confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        confusion_path = "confusion_matrix.png"
        plt.savefig(confusion_path)
        plt.close()

        # Log confusion matrix image
        if os.path.exists(confusion_path):
            mlflow.log_artifact(confusion_path)
        else:
            print("⚠️ confusion_matrix.png not found!")

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Run experiments
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    try:
        print(f"Running experiment for max_features={max_features}...")
        run_experiment_tfidf_max_features(max_features)
        time.sleep(3)  # Optional: wait to avoid overloading MLflow server
    except Exception as e:
        print(f"❌ Error for max_features={max_features}: {e}")

Running experiment for max_features=1000...


2025/07/15 17:14:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 17:15:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/dfcc33d09dfd495fb074f3cb98885707
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513
Running experiment for max_features=2000...


2025/07/15 17:15:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 17:16:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/867d36e23d9947789903e3a17108ed92
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513
Running experiment for max_features=3000...


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/aa4e9ea7e28e4394ad35ecb7862d16b0
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513
Running experiment for max_features=9000...


2025/07/15 17:22:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 17:22:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/145460990cf6432992133279502b14c7
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513
Running experiment for max_features=10000...


2025/07/15 17:22:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 17:23:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/fda7331b125544f8a841b6ddd82f8a87
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513


In [16]:
# Setup
import os
os.environ["MLFLOW_HTTP_REQUEST_TIMEOUT"] = "400"

import mlflow
mlflow.set_tracking_uri("http://ec2-18-215-180-118.compute-1.amazonaws.com:5000")

# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        confusion_path = "confusion_matrix.png"  # 🔧 Add this
        plt.savefig(confusion_path)
        if os.path.exists(confusion_path):
            mlflow.log_artifact(confusion_path)
        else:
            print("⚠️ confusion_matrix.png not found!")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/07/15 22:53:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 22:53:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/101215f0dee541d38408c0af9e13d5aa
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513


2025/07/15 22:55:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 22:55:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/17c59a9992684a5795a5644de15108e2
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513


2025/07/15 22:57:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 22:57:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/1491229642f341ebbce9d2876bbb70bb
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513


2025/07/15 23:07:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 23:08:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/ddfc5db50da043bd8484201eaaaa4a73
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513


2025/07/15 23:09:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 23:16:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513/runs/98dce96378404936a3e94daf7d11f38b
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/572389916107840513
